In [1]:
#!pip install langchain

In [2]:
#%pip install --upgrade --quiet unstructured



In [3]:
#import nltk
#nltk.download('punkt')


In [4]:
from langchain_community.document_loaders import UnstructuredURLLoader  

In [5]:
#%pip install unstructured libmagic python-magic python-magic-bin

In [6]:
loader=UnstructuredURLLoader(urls=[
    'https://www.cbsl.gov.lk/en/financial-system/financial-markets/government-securities-market',
    'https://www.sc.com/ke/investments/learn/understanding-bonds-for-beginners/'
])

In [7]:
pip show unstructured


Name: unstructuredNote: you may need to restart the kernel to use updated packages.

Version: 0.15.13
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: c:\Python312\Lib\site-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, langdetect, lxml, nltk, numpy, psutil, python-iso639, python-magic, python-oxmsg, rapidfuzz, requests, tabulate, tqdm, typing-extensions, unstructured-client, wrapt
Required-by: 


In [13]:
data=loader.load()

In [14]:
len(data)

2

In [15]:
data[0]

Document(metadata={'source': 'https://www.cbsl.gov.lk/en/financial-system/financial-markets/government-securities-market'}, page_content="Skip to main content\n\nEnglish\n\nසිංහල\n\nதமிழ்\n\nNavigation\n\nABOUT\n\nAbout the Bank\n\nOverview\n\nHistory\n\nVision, Mission & Values\n\nObjectives\n\nFunctions\n\nOrganisational Structure\n\nCorporate Structure\n\nPrincipal Officers\n\nDepartments\n\nBank Premises\n\nBank Headquarters\n\nRegional Offices\n\nLibrary & Information Center\n\nCentre For Banking Studies\n\nThe Economic History Museum of Sri Lanka\n\nExter Report\n\nMONETARY POLICY\n\nAbout Monetary Policy\n\nOverview\n\nEconomic and Price Stability\n\nMonetary Policy Framework\n\nInstruments & Implementation\n\nPolicy Rates and Open Market Operations\n\nStatutory Reserve Requirement\n\nOther Policy Instruments\n\nMonetary Policy Committee\n\nStakeholder Engagement Committee\n\nMonetary Policy Communication\n\nOverview\n\nMonetary Policy Announcement Dates\n\nMonetary Policy Advan